In [6]:
#my investment calculator parameters
import math

downpayment = 0
dividend = 1000

bank = downpayment
depositTotal = downpayment
depositTotalFuture = downpayment
interestCycle = 'quarterly'

rate = 1.05
inflation = 1.05

years = 40
months = years * 12

def refreshBank():
    global bank, depositTotal, depositTotalFuture
    bank = downpayment
    depositTotal = downpayment
    depositTotalFuture = downpayment
    
def monthIncrease():
    global bank, depositTotal, depositTotalFuture
    bank += dividend
    depositTotal += dividend
    depositTotalFuture += dividend
    
def interval(period='annually'):
    if period == 'monthly':
        eachTime = 1
    elif period == 'bimonthly':
        eachTime = 2
    elif period == 'quarterly':
        eachTime = 3
    elif period == 'semi':
        eachTime = 6
    elif period == 'yearly' or period == 'annually':
        eachTime = 12
    else:
        print("The interest interval was not defined. Default to annually.")
        eachTime = 1
    return eachTime

def roundToCents(money, floorQ=False, intergerQ=True):
    if isinstance(money, float):
        if floorQ:
            rounded = math.floor(money*1e2) / 1e2
            #rounded = money // 0.01 / 100
        else:
            rounded = round(money, 2)
        ans = rounded
    elif isinstance(money, int):
        ans = money
    ans = ans if intergerQ else str(ans)
    return ans
    

In [7]:
#investment calculator
refreshBank()

interestCycle = 'annually'
eachTime = interval(interestCycle)

for month in range(months):
    monthIncrease()
    #interest increase if interest period reached
    if (month % eachTime) + 1 == 1:
        bank *= rate
    #inflation devalue if end of year
    if (month % 12) + 1 == 1:
        bank /= inflation
        depositTotalFuture /= inflation

gains = bank - depositTotalFuture

bankRounded = roundToCents(bank)
gainsRounded = roundToCents(gains)

print(f"Final bank, ${bankRounded}")
print(f"Investment gain: ${gainsRounded}")
print(f"Total deposited: {depositTotal}")

Final bank, $716899.0
Investment gain: $431893.86
Total deposited: 480000


In [9]:
#how long to get certain amount of money

refreshBank()
month = 0
#target = 1.5e6
target = 450e3
bank = 0

bankProgress = target
loseCount = 0
loseCountLimit = 3

#interestCycle = 'quarterly'
interestCycle = 'yearly'
eachTime = interval(interestCycle)


while True:
    monthIncrease()
    
    if (month % eachTime) + 1 == 1:
        bank *= rate
        
    if (month % 12) + 1 == 1:
        bank /= inflation
    
    #if bank not getting closer, break
    tempBankProgress = target - bank
    loseCount = loseCount + 1 if tempBankProgress > bankProgress else 0
    if loseCount > loseCountLimit:
        print(f"The bank is not getting bigger. This is the bank: {bank}.")
        break
    else:
        bankProgress = target - bank
        
    if bank >= target:
        break
        
    #iterate
    month += 1
        
yea, mon = divmod(month, 12)
if yea:
    strung = f"It took {yea} years and {mon} months to get ${target}."
else:
    strung = f"It took {month} months to get ${target}."
    
print(strung)


It took 7 years and 5 months to get $450000.0.


In [93]:
#money market income

#option input
#bank = 450e3
bank = 1e5

moneyMarketRate = rate - 1
moneyMarketPeriod = interval(interestCycle)

periodSalary = bank * moneyMarketRate
monthlySalary = periodSalary / moneyMarketPeriod
yearlySalary = monthlySalary * 12
salary = monthlySalary

print(f"Bank: ${bank}, Salary: ${salary}.")

Bank: $100000.0, Salary: $1333.3333333333346.


In [92]:
#money market bank (from salary)

salary = 6000
salary = 4000
moneyMarketRate = rate - 1
moneyMarketPeriod = interval(interestCycle)

periodSalary = salary * moneyMarketPeriod
moneyMarketBank = periodSalary / moneyMarketRate

print(f"For the salary ${salary}, you would need a bank of ${moneyMarketBank} in the money market fund.")

For the salary $4000, you would need a bank of $299999.9999999997 in the money market fund.


In [66]:
#keeping up to date on republic bank mutual fund

import time, configparser, re, requests, sqlite3
from bs4 import BeautifulSoup


#Republic Caribbean Equity Fund
def republic_caribbean_equity_fund():
    #GET NUMBERS OFF OF WEBSITE
    #get web info
    url = 'https://republictt.com/corporate/republic-mutual-funds'
    html = requests.get(url).text
    soup = BeautifulSoup(html)
    
    #parse web info
    #hierarchy: tbody > tr > td > (all the numbers)
    match_strings = []
    escape_bool = False
    for tbody in soup.find_all('tbody'):
        #print(tbody, '\n')
        for tr in tbody.find_all('tr'):
            elements = [td.string for td in tr.find_all('td')]
            match_strings = [re.search(r'[-+]?[0-9]*\.?[0-9]*', ele).group(0) for ele in elements if isinstance(ele,str)]
            if all(match_strings):
                escape_bool = True
                #print(match_strings)
            if escape_bool: break        
        if escape_bool: break
            
    #SAVE NUMBERS
    #using sqlite
    conn = sqlite3.connect('financial info.db')
    #conn = sqlite3.connect(':memory:')
    sql_values = [time.time(),] + [str(match_strings[i]) for i in [0,1,2,-1]]
    with conn:
        #create table if it doesnt exist
        conn.execute('''CREATE TABLE IF NOT EXISTS Caribbean_Equity_Fund_Trends
        (date REAL , offer_price real, bid_price real, ytd_return real, cummulative_from_inception real);''')
        
        prompt = input(f"Do you accept these numbers? [Y/N]\n{sql_values}\n")
        if 'y' in prompt.lower():
            #add values to table
            print('You accept these values.')
            conn.execute('''INSERT INTO Caribbean_Equity_Fund_Trends 
            VALUES (?,?,?,?,?);''', sql_values)
        else:
            print('You do not accept these values.')
        
        #delete all database values
        #conn.execute('DELETE FROM Caribbean_Equity_Fund_Trends;')
        
        #print values of table
        maco = True
        if maco:
            print("This is the current stores values.")
            maco = conn.execute('SELECT * FROM Caribbean_Equity_Fund_Trends;')
            for r in maco:
                print(r)
    conn.close()
republic_caribbean_equity_fund()


Do you accept these numbers? [Y/N]
[1604878270.719397, '60.8690', '59.6755', '-17.24', '381.23']
n
You do not accept these values.
This is the current stores values.
(1604877793.0730772, 60.869, 59.6755, -17.24, 381.23)


In [44]:
import datetime
dir(datetime)
#datetime.datetime.now()
time.time()

1604876811.2889225